Даже с батчем размера 1 не влезает...
- 391 321 600 параметров

<img src="./img/prophet.jpg" width="300">





- из данных убраны дубли

In [2]:
TRAIN_CSV = f"./datasets/train_clean.csv"
SMALL_CSV = f"./cache/train.csv"
SCORING_CSV = f"./datasets/test.csv"

USE_SMALL = True

In [3]:
import torch
from tqdm import tqdm

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Датасеты

- пакет huggingface datasets

In [4]:
# !pip install datasets
import datasets

In [5]:
arxiv_dataset = datasets.Dataset.from_csv(SMALL_CSV if USE_SMALL else TRAIN_CSV)

Using custom data configuration default-04a3546adcf83ebb
Reusing dataset csv (/home/user1/.cache/huggingface/datasets/csv/default-04a3546adcf83ebb/0.0.0)


In [6]:
test_size = 0.2 if USE_SMALL else 0.02
arxiv_dataset = arxiv_dataset.train_test_split(test_size=test_size)
pass

In [7]:
len(arxiv_dataset["train"]), len(arxiv_dataset["test"]), arxiv_dataset["train"][0].keys()

(800, 200, dict_keys(['abstract', 'title']))

In [8]:
scoring_dataset = datasets.Dataset.from_csv(SCORING_CSV)
len(scoring_dataset), scoring_dataset[0].keys()

Using custom data configuration default-2e0a9ad90b647d2d
Reusing dataset csv (/home/user1/.cache/huggingface/datasets/csv/default-2e0a9ad90b647d2d/0.0.0)


(1000, dict_keys(['abstract']))

## Токенайзер

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/prophetnet-large-uncased")   # 0.2 MB

/home/user1/envs/py310/lib64/python3/site-packages/transformers/configuration_utils.py:358: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [10]:
prefix = "summarize: "

def preprocess_function(examples):
    """ Длины последовательностей max = 1096 / 103. Берем какбе 3 сигмы:
        - max_length=457 + 3
        - max_length=34 + 2
    """

    srcs = [prefix + doc for doc in examples["abstract"]]
    model_inputs = tokenizer(srcs, max_length=460, truncation=True)

    with tokenizer.as_target_tokenizer():
        trgs = tokenizer(examples["title"], max_length=36, truncation=True)

    model_inputs["labels"] = trgs["input_ids"]
    return model_inputs

In [11]:
tokenized_arxiv = arxiv_dataset.map(preprocess_function, batched=True)

100%|██████████| 1/1 [00:00<00:00,  1.43ba/s]


In [12]:
tokenized_arxiv["train"][0].keys(), tokenized_arxiv["train"][0]["abstract"], tokenized_arxiv["train"][0]["title"]

(dict_keys(['abstract', 'title', 'input_ids', 'attention_mask', 'labels']),
 "exploiting relationships between visual regions and question words have achieved great success in learning multi-modality features for visual question answering (vqa). however, we argue that existing methods mostly model relations between individual visual regions and words, which are not enough to correctly answer the question. from humans' perspective, answering a visual question requires understanding the summarizations of visual and language information. in this paper, we proposed the multi-modality latent interaction module (mli) to tackle this problem. the proposed module learns the cross-modality relationships between latent visual and language summarizations, which summarize visual regions and question into a small number of latent representations to avoid modeling uninformative individual region-word relations. the cross-modality information between the latent summarizations are propagated to fuse va

# Модель

In [13]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/prophetnet-large-uncased") # 1460MB

/home/user1/envs/py310/lib64/python3/site-packages/transformers/configuration_utils.py:358: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [14]:
from helpers.utils import count_parameters

count_parameters(model)

+-----------------------------------------------------------------------+-----+--------------+------------+---------------+-----------+
|                            Modules/Tensors                            | GPU |    Shape     | Parameters |      Type     |  DataMem  |
+-----------------------------------------------------------------------+-----+--------------+------------+---------------+-----------+
|                   prophetnet.word_embeddings.weight                   |     | 30522 x 1024 |  31254528  | torch.float32 | 125018112 |
|             prophetnet.encoder.position_embeddings.weight             |     |  512 x 1024  |   524288   | torch.float32 |  2097152  |
|            prophetnet.encoder.embeddings_layer_norm.weight            |     |     1024     |    1024    | torch.float32 |    4096   |
|             prophetnet.encoder.embeddings_layer_norm.bias             |     |     1024     |    1024    | torch.float32 |    4096   |
|         prophetnet.encoder.layers.0.self_attn.

391321600

In [1]:
model.to(device)    # if no train else trainer sends

NameError: name 'model' is not defined

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Обучение

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./Prophet-large-results",
    optim="adamw_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    weight_decay=0.01,
    # logging_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    num_train_epochs=3*16,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_arxiv["train"],
    eval_dataset=tokenized_arxiv["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
tqdm._instances.clear()

# trainer.train()#(resume_from_checkpoint=True)

# Генерация

In [ ]:
def generate(example):
    input_ids = tokenizer(prefix + example["abstract"], 
                        max_length=460, 
                        truncation=True, 
                        return_tensors="pt").input_ids  # Batch size 1
    outputs = model.generate(input_ids.to(device))
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
n = 10
arxiv_dataset["test"][n]["abstract"], arxiv_dataset["test"][n]["title"], generate(arxiv_dataset["test"][n])

In [ ]:
n = 42
arxiv_dataset["test"][n]["abstract"], arxiv_dataset["test"][n]["title"], generate(arxiv_dataset["test"][n])

# BLEU-score

Самоделки:
- 0.02457 (словарь 6152, по 5 эпох по 5r-4, 1e-3, min.val.loss = 3.875) 
- **0.19204** (словарь 60 тыс. ~15 эпох с шагом 5e-4 -> 5e-5, min.val.loss = 2.289)
- 0.12601 (словарь 84 тыс. много разных эпох, сходится плохо, min.val.loss = 3.305)
- 0.10644 (BPE, словарь 16 тыс., много разных эпох, сходится плохо, min.val.loss = 3.8)

T5-small
- BLEU-score: **0.044...** 1% тюнинг
- BLEU-score: **0.16563** (3 эпохи - 2,5 часа RTX2060 6Gb)

T5-base
- BLEU-score: **0.07422** (без обучения)
- обучение не тянет...

BART-base
- BLEU-score: **0.17743** 1% тюнинг
- BLEU-score: **0.17984** (1.43 эпохи - 2,5 часа RTX2060 6Gb)
- BLEU-score: **0.19266** (2 эпохи)


In [ ]:
from torchtext.data.metrics import bleu_score

tqdm._instances.clear()

candidates = []
references = []
for example in tqdm(tokenized_arxiv["test"]):
    candidates.append(generate(example).split())
    references.append([example["title"].split()])

score = bleu_score(candidates, references, max_n=3, weights=[1/3]*3)

print('BLEU-score: {0:.5f}'.format(score))

### Stepik score

In [ ]:
SUBMISSION_NAME = "Prophet-large" if USE_SMALL else "Prophet-large-tune"

Генерация заголовков для тестовых данных

In [ ]:
tqdm._instances.clear()

abstracts = []
titles = []

for example in tqdm(scoring_dataset):
    abstracts.append(example["abstract"])
    titles.append(generate(example))

Получилось, например

In [ ]:
abstracts[1], titles[1]

Записываем полученные заголовки в файл формата `<abstract>,<title>`:

In [ ]:
import pandas as pd

submission_df = pd.DataFrame({'abstract': abstracts, 'title': titles})
submission_df.to_csv(f"./submission/predicted_titles_{SUBMISSION_NAME}.csv", index=False)

In [ ]:
submission_df["title"].apply(lambda x: len(str(x).split())).describe()[["mean","std", "max"]]

С помощью скрипта `generate_csv` приводим файл `submission_prediction.csv` в формат, необходимый для отправки:

In [ ]:
from helpers.create_submission import generate_csv

generate_csv(input_file=f"./submission/predicted_titles_{SUBMISSION_NAME}.csv", 
             output_file=f'./submission/submission_{SUBMISSION_NAME}.csv', 
             voc_file=f'./datasets/vocs.pkl')

# С учетом

In [ ]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("./datasets/train.csv")
submission_df = pd.read_csv(f"./submission/predicted_titles_{SUBMISSION_NAME}.csv")

intersect_idx = np.intersect1d(submission_df["abstract"].str.lower(), train_df["abstract"].str.lower(), return_indices=True)

submission_df.loc[intersect_idx[1], 'title'] = train_df.loc[intersect_idx[2], 'title'].values

In [ ]:
from helpers.create_submission import generate_csv

submission_df.to_csv(f"./submission/predicted_titles_{SUBMISSION_NAME}_fake.csv", index=False)

generate_csv(input_file=f"./submission/predicted_titles_{SUBMISSION_NAME}_fake.csv", 
             output_file=f'./submission/submission_{SUBMISSION_NAME}_fake.csv', 
             voc_file=f'./datasets/vocs.pkl')

In [ ]:
f'./submission/submission_{SUBMISSION_NAME}_fake.csv'

T5-small:
- **Score: 0.26174** 1% tuning
- **Score: 0.34497** tuning 3 эпохи
- **Score: 0.51810** + добавление правильных меток из трейна

T5-base:
- **Score: 0.20510** w/o tuning
- для обучения с имеющейся длиной последовательности не хватает памяти GPU

BART-base
- **Score: 0.33851** 1% tuning
- **Score: 0.39536** tuning 1,5 эпохи
- **Score: 0.54804** + добавление правильных меток из трейна
- **Score: 0.56782** 2 эпохи с накопление градиента (вот и в топ-10)
- ... дальше не интересно